In [1]:
import argparse
import datetime
import numpy as np

from tensorflow.keras.callbacks import ModelCheckpoint, History, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.layers import BatchNormalization, Input, Reshape, Dense, LSTM, Dropout, Permute
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from timeit import default_timer as timer
from src.train.utils import build_numpy, build_generator, fetch_file_list_pn, BATCH_SIZE, build_numpy, \
    fetch_file_list


X_COLUMNS = 1
Y_COLUMNS = 0
BATCH_SIZE = 128
LR = 5e-4
# number of samples. A tiny number of samples for fast test
# For real training should reflect number of samples in whole training set
NUM_TRAIN_SAMPLES = BATCH_SIZE * 2
NUM_VALI_SAMPLES = BATCH_SIZE * 2

EPOCHS = 5
np.random.seed(100)

TRAINING_DATA_DIR = '/Users/lucindazhao/strava/ml-local/trainData/'
VALIDATION_DATA_DIR = '/Users/lucindazhao/strava/ml-local/validationData/'
TRAIN_PORTION = 10.1
VALIDATION_PORTION = 10.1
MODEL_CHECKPOINT = '/Users/lucindazhao/strava/ml-local/snapshots/v1/'
LOG_DIR = '/Users/lucindazhao/strava/ml-local/logs/'
IS_CSV = False


Using TensorFlow backend.


In [2]:
def build_lstm(lr):
    # build CNN network
    # model definition
    modelGraph = Sequential()
    # make sure to define input layer
    input1 = Input(shape=(None,))
    # get N * 26 input matrix
    reshape2 = Reshape((26, -1), name="reshape2")

    # get 26 * N input matrix
    transpose = Permute((2, 1), name="transpose")
    layer1 = LSTM(units=200, return_sequences = True, name="lstm1")
    layer2 = LSTM(units=200, return_sequences = False, name="lstm2")
    dropout = Dropout(0.3, name="dropout")
    dense_layer = Dense(units=2, activation="softmax", kernel_regularizer=regularizers.l2(1e-3))
    modelGraph.add(input1)
    modelGraph.add(reshape2)
    modelGraph.add(transpose)
    modelGraph.add(layer1)
    modelGraph.add(layer2)
    modelGraph.add(dropout)
    modelGraph.add(dense_layer)
    
    print(modelGraph.summary())
    return 'lstm_200hidden_2layer_lr' + str(lr), modelGraph

In [3]:
build_lstm(0.01)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape2 (Reshape)           (None, 26, None)          0         
_________________________________________________________________
transpose (Permute)          (None, None, 26)          0         
_________________________________________________________________
lstm1 (LSTM)                 (None, None, 200)         181600    
_________________________________________________________________
lstm2 (LSTM)                 (None, 200)               320800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 402       
Total params: 502,802
Trainable params: 502,802
Non-trainable params: 0
__________________________________________________

('lstm_200hidden_2layer_lr0.01',
 <tensorflow.python.keras.engine.sequential.Sequential at 0x14e319a58>)

# Train Mini model first

In [4]:
train_file_list = fetch_file_list(data_dir=TRAINING_DATA_DIR, portion=TRAIN_PORTION)
tg = build_numpy(file_list=train_file_list, num_samples=None, xcolumns=X_COLUMNS, shuffle=False,
                 ycolumns=Y_COLUMNS, ytx=None, is_csv=IS_CSV)

val_file_list = fetch_file_list(data_dir=VALIDATION_DATA_DIR, portion=VALIDATION_PORTION)
vg = build_numpy(file_list=val_file_list, num_samples=None, xcolumns=X_COLUMNS, ycolumns=Y_COLUMNS, ytx=None,
                 skip_header=1, shuffle=False, is_csv=IS_CSV)

count = np.sum(tg[1], axis=0)
print("Number of Positive Training Windows: {}".format(count[0]))
print("Number of Negative Training Windows: {}".format(len(tg[1]) - count[0]))

Number of Files: 10
Number of Files: 10
Number of Positive Training Windows: 192986.0
Number of Negative Training Windows: 2547081.0


In [ ]:
for lr in [5e-4, 1e-4, 1e-3, 1e-2]:
    model_pair = build_lstm(lr)
    key = model_pair[0] + '_small'
    model = model_pair[1]
    print(key)
    print(model.summary())
    file_path = MODEL_CHECKPOINT + key + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    history = History()
    log_dir = LOG_DIR + key + '/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensor_board = TensorBoard(log_dir, histogram_freq=5,
                               write_grads=False, write_graph=False)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                          patience=4, min_lr=1e-5)
    checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=5)

    # omit reduce_lr for now
    callbacks_list = [history, tensor_board, checkpoint]

    adam_wn = Adam(lr=lr)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=adam_wn, metrics=['accuracy'])
    model.fit(x=tg[0], y=tg[1], validation_data=vg, batch_size=BATCH_SIZE,
      epochs=EPOCHS, verbose=1, shuffle=False,
      callbacks=callbacks_list, class_weight={0: 0.1, 1: 1.0})
    model.save(file_path)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape2 (Reshape)           (None, 26, None)          0         
_________________________________________________________________
transpose (Permute)          (None, None, 26)          0         
_________________________________________________________________
lstm1 (LSTM)                 (None, None, 200)         181600    
_________________________________________________________________
lstm2 (LSTM)                 (None, 200)               320800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 502,802
Trainable params: 502,802
Non-trainable params: 0
________________________________________________